![CAO](https://upload.wikimedia.org/wikipedia/commons/5/51/Central_Applications_Office.png)

# CAO Points Analysis

http://www.cao.ie/index.php?page=points&p=2021
***

In [1]:
# Regular expressions.
import re

# Convenient HTTP requests.
import requests as rq

# Import Pandas
import pandas as pd

# Dates and times.
import datetime as dt

#import csv
import csv

#import camelot
import camelot

# For downloading.
import urllib.request as urlrq

In [2]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Have a quick peek. 200 means OK.
resp

<Response [200]>

In [3]:
# Get the current date and time.
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Create a file path for the original data.
path2021 = 'data/cao2021_' + nowstr + '.html'

In [5]:

# The server uses the wrong encoding, fix it.
original_encoding = resp.encoding

# Change to cp1252.
resp.encoding = 'cp1252'

In [6]:
# Save the original html file.
with open(path2021, 'w') as f:
    f.write(resp.text)

Compile the regular expression for matching lines. r' means raw string notation. [01] This regular expression means that we are looking for 2
characer set of upper case letters from A-Z. The {}'s denote how many
occurnaces of the character set we are looking for. This is followed by 3
groups. Groups allow us to match several different patterns. The first group
is any character except a newline (.) with 0 or more repetitions (*). The
second group is 3 digits (\d{3}). The third group is a literal * which is
optional. The ? denotes that it is optional.

In [7]:
re_course = re.compile(r'([A-Z]{2}\d{3})  (.*)')

### Loop through the lines of the response
***

In [8]:
# The file path for the csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [10]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [11]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


### Cleaning up the Data

In [12]:
df2021.iloc[842]

code                                                    MH801
title       Early Childhood - Teaching and Learning (part-...
pointsR1                                                  AQA
pointsR2                                                  AQA
Name: 842, dtype: object

In [13]:
#  [11]

df2021.replace('[*, #]', '', regex = True, inplace = True)
df2021.replace('AQA', '', regex = True, inplace = True)
df2021

,code,title,pointsR1,pointsR2
0,AL801,SoftwareDesignforVirtualRealityandGaming,300,NaN
1,AL802,SoftwareDesigninArtificialIntelligenceforClo,313,NaN
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi,350,NaN
3,AL805,ComputerEngineeringforNetworkInfrastructure,321,NaN
4,AL810,QuantitySurveying,328,NaN
...,...,...,...,...
944,WD211,CreativeComputing,270,NaN
945,WD212,RecreationandSportManagement,262,NaN
946,WD230,MechanicalandManufacturingEngineering,230,230
947,WD231,EarlyChildhoodCareandEducation,266,NaN


In [14]:
df2021.iloc[842]

code                                               MH801
title       EarlyChildhood-TeachingandLearning(part-time
pointsR1                                                
pointsR2                                                
Name: 842, dtype: object

## Data Types

In [15]:
# Check the data types [10]
df2021.dtypes

code        object
title       object
pointsR1    object
pointsR2    object
dtype: object

In [16]:
# change the points columns from object to float [09]

df2021['pointsR1'] = pd.to_numeric(df2021['pointsR1'])
df2021['pointsR2'] = pd.to_numeric(df2021['pointsR2'])
df2021

,code,title,pointsR1,pointsR2
0,AL801,SoftwareDesignforVirtualRealityandGaming,300.0,NaN
1,AL802,SoftwareDesigninArtificialIntelligenceforClo,313.0,NaN
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi,350.0,NaN
3,AL805,ComputerEngineeringforNetworkInfrastructure,321.0,NaN
4,AL810,QuantitySurveying,328.0,NaN
...,...,...,...,...
944,WD211,CreativeComputing,270.0,NaN
945,WD212,RecreationandSportManagement,262.0,NaN
946,WD230,MechanicalandManufacturingEngineering,230.0,230.0
947,WD231,EarlyChildhoodCareandEducation,266.0,NaN


In [17]:
df2021.dtypes

code         object
title        object
pointsR1    float64
pointsR2    float64
dtype: object

# Save clean data

In [18]:
# Create a file path for the pandas data
path2021clean = 'data/cao2021_clean_csv_' + nowstr + '.csv'

In [19]:
# Save pandas data frame to disk
df2021.to_csv(path2021clean) 

<br>

## 2020 Points

https://www.cao.ie/index.php?page=points&p=2020
***

In [20]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

**Save Original File**

In [21]:
# Create a file path for the original data.
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

In [22]:
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_20211222_203731.xlsx',
 <http.client.HTTPMessage at 0x1cfede49790>)

## Load Spreadsheet using pandas

In [23]:
# Download and parse the excel spreadsheet.
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [24]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Spot check a random row.
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [26]:
# Spot check the last row.
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [27]:
# Create a file path for the pandas data.
path = 'data/cao2020_' + nowstr + '.csv'

In [28]:
# Save pandas data frame to disk.
df2020.to_csv(path)

In [29]:
df2020 = df2020.loc[df2020["LEVEL"] > 7]
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Arts,Arts (options),WD200,AQA,NaN,AQA,NaN,AQA,NaN,336,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

## 2019 Points
https://www.cao.ie/index.php?page=points&p=2019
***

**_Step to reproduce_**
<br>

1. Download and Install [Ghostscript](https://www.ghostscript.com/releases/gsdnld.html). Choose public license 64 bit.
2. Install [Camelot](https://camelot-py.readthedocs.io/en/master/) using the command ``conda install -c conda-forge camelot-py`` in cmder.
3. Import the Camelot package using the command ``import camelot`` as seen in the import cell at the top of this project.
4. Create a **[TableList](https://camelot-py.readthedocs.io/en/master/api.html#camelot.core.TableList)** object called ``tables``, which is a list of Table objects using the file path of the 2019 data as seen below.
5. Seince there are 18 pages in this PDF, Set ``the read_pdf`` parameter as a range from 1-18 

In [30]:
tables = camelot.read_pdf('data/cao2019_20211104_143030.pdf', pages = '1-18')

In [31]:
tables

<TableList n=18>

Above, we can see that the tables object has 18 tables, since n=18.

In [32]:
type(tables)

camelot.core.TableList

To access a table by using its index. We can select a table passing the index.

In [33]:
tables[0]

<Table shape=(44, 4)>

The above is the first table on the PDF file. We can also see the shape of the table, 4 rows and 44 columns. The next thing to do is to make an object for each table index below so that we can use pandas to concatonate all of the tables.

In [34]:
table_1 = tables[0]
table_2 = tables[1]
table_3 = tables[2]
table_4 = tables[3]
table_5 = tables[4]
table_6 = tables[5]
table_7 = tables[6]
table_8 = tables[7]
table_9 = tables[8]
table_10 = tables[9]
table_11 = tables[10]
table_12 = tables[11]
table_13 = tables[12]
table_14 = tables[13]
table_15 = tables[14]
table_16 = tables[15]
table_17 = tables[16]
table_18 = tables[17]

Let's take a look at the first and last tables using the df function in Camelot.

In [35]:
table_1.df

,0,1,2,3
0,Course Code INSTITUTION and COURSE,,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
7,AL820,Mechanical and Polymer Engineering,300,358
8,AL830,General Nursing,410,429
9,AL832,Psychiatric Nursing,387,403


In [36]:
table_18.df

,0,1,2,3
0,TL881,Social Care,308,356
1,TL889,Counselling with Addiction - Mature applicants...,#,
2,TL890,General Nursing and Mature Applicants,407,431
3,TL891,Mental Health Nursing and Mature Applicants,367*,379
4,,Waterford Institute of Technology,,
5,WD001,Applied Computing (options),279,358
6,WD002,Science (options),302,377
7,WD005,Health Sciences (options),288,347
8,WD006,Exercise Sciences (options),282,337
9,WD007,Engineering (options),287,382


We need to drop the rows that are not course names (Name of the college and the first row in table_1 which is just the headers which we will write ourselves later. To do this, take a look at each table and find the indexes that need to be dropped.for table_1 we need to drop indexes 0, 1 and 29. I will just show how the first table because the process just needs to be repeated for the rest of the tables.

In [37]:
# Dropped the indexes that we do not need
table_1.df.drop([0, 1, 29])

,0,1,2,3
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
7,AL820,Mechanical and Polymer Engineering,300,358
8,AL830,General Nursing,410,429
9,AL832,Psychiatric Nursing,387,403
10,AL836,Nutrition and Health Science,352,383
11,AL837,Sports Science with Exercise Physiology,351,392


Once we know which rows to drop we then need to make the drop operations that we just performed permanent since these operations are not inplace. To do this we need to perform the operation for each table and then assign the operations back to the original variables as seen below. We could use the ``inplace = True`` option of the function, but this example it is neater to assign the operations back to the original variable i.e. the tables.

In [38]:
# Assign the drop operations back to the original variables.
table_1 = table_1.df.drop([0, 1, 29])
table_2 = table_2.df.drop([19, 23])
table_3 = table_3.df.drop([12])
table_4 = table_4.df.drop([20, 23, 25, 32])
table_5 = table_5.df.drop([1])
table_6 = table_6.df.drop([12])
table_7 = table_7.df.drop([27, 45])
table_8 = table_8.df.drop([9, 13, 32])
table_9 = table_9.df.drop([2, 13, 22, 27])
table_10 = table_10.df.drop([30])
table_11 = table_11.df.drop([17, 40])
table_12 = table_12.df.drop([33])
table_13 = table_13.df.drop([45])
table_14 = table_14.df.drop([10, 24, 22])
table_15 = table_15.df.drop([21])
table_16 = table_16.df.drop([8, 11, 47])
table_17 = table_17.df.drop([0, 33])
table_18 = table_18.df.drop([4])

We can now make an dataframe using the concatonate function in pandas.

In [39]:
df2019 = pd.concat([table_1, table_2, table_3, table_4, table_5,
                    table_6, table_7, table_8, table_9, table_10,
                    table_11, table_12, table_13, table_14, table_15,
                    table_16, table_17, table_18], ignore_index = True)

In [40]:
df2019.columns = ['code','title', 'points_r1', 'points_r2']
df2019

,code,title,points_r1,points_r2
0,AL801,Software Design with Virtual Reality and Gaming,304,328
1,AL802,Software Design with Cloud Computing,301,306
2,AL803,Software Design with Mobile Apps and Connected...,309,337
3,AL805,Network Management and Cloud Infrastructure,329,442
4,AL810,Quantity Surveying,307,349
...,...,...,...,...
925,WD200,Arts (options),221,296
926,WD210,Software Systems Development,271,329
927,WD211,Creative Computing,275,322
928,WD212,Recreation and Sport Management,274,311


In [41]:
df2019['points_r2'].dtype

dtype('O')

## Concatonate and Join

In [42]:
courses2021 = df2021[['code', 'title']]
courses2021

,code,title
0,AL801,SoftwareDesignforVirtualRealityandGaming
1,AL802,SoftwareDesigninArtificialIntelligenceforClo
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi
3,AL805,ComputerEngineeringforNetworkInfrastructure
4,AL810,QuantitySurveying
...,...,...
944,WD211,CreativeComputing
945,WD212,RecreationandSportManagement
946,WD230,MechanicalandManufacturingEngineering
947,WD231,EarlyChildhoodCareandEducation


In [43]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1455,WD200,Arts (options)
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [44]:
courses2019 = df2019[['code', 'title']]
courses2019

,code,title
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying
...,...,...
925,WD200,Arts (options)
926,WD210,Software Systems Development
927,WD211,Creative Computing
928,WD212,Recreation and Sport Management


In [45]:
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allcourses

,code,title
0,AL801,SoftwareDesignforVirtualRealityandGaming
1,AL802,SoftwareDesigninArtificialIntelligenceforClo
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi
3,AL805,ComputerEngineeringforNetworkInfrastructure
4,AL810,QuantitySurveying
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [46]:
allcourses.sort_values('code')

,code,title
175,AC120,InternationalBusiness
2144,AC120,International Business
949,AC120,International Business
176,AC137,LiberalArts
2145,AC137,Liberal Arts
...,...,...
1975,WD230,Mechanical and Manufacturing Engineering
946,WD230,MechanicalandManufacturingEngineering
2905,WD230,Mechanical and Manufacturing Engineering
947,WD231,EarlyChildhoodCareandEducation


In [47]:
allcourses.loc[175]['title']

'InternationalBusiness'

In [48]:
allcourses.loc[949]['title']

'International Business'

In [49]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

,code,title
972,AL838,Biotechnology
973,AL839,Microbiology
978,AL850,Business
980,AL852,Accounting
1016,CK123,Anthropology
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [50]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

,code,title
0,AL801,SoftwareDesignforVirtualRealityandGaming
1,AL802,SoftwareDesigninArtificialIntelligenceforClo
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi
3,AL805,ComputerEngineeringforNetworkInfrastructure
4,AL810,QuantitySurveying
...,...,...
2845,TL802,"TV, Radio and New Media"
2846,TL803,Music Technology
2849,TL812,Computing with Digital Media
2853,TL842,Construction Management


In [51]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [52]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [53]:
allcourses

,code,title
0,AL801,SoftwareDesignforVirtualRealityandGaming
1,AL802,SoftwareDesigninArtificialIntelligenceforClo
2,AL803,SoftwareDesignforMobileAppsandConnectedDevi
3,AL805,ComputerEngineeringforNetworkInfrastructure
4,AL810,QuantitySurveying
...,...,...
1209,SG441,Environmental Science
1210,SG446,Applied Archaeology
1211,TL803,Music Technology
1212,TL812,Computing with Digital Media


## Join the Points

In [54]:
# Set the index to the code column.
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,SoftwareDesignforVirtualRealityandGaming,300.0,NaN
AL802,SoftwareDesigninArtificialIntelligenceforClo,313.0,NaN
AL803,SoftwareDesignforMobileAppsandConnectedDevi,350.0,NaN
AL805,ComputerEngineeringforNetworkInfrastructure,321.0,NaN
AL810,QuantitySurveying,328.0,NaN
...,...,...,...
WD211,CreativeComputing,270.0,NaN
WD212,RecreationandSportManagement,262.0,NaN
WD230,MechanicalandManufacturingEngineering,230.0,230.0


In [55]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [56]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

,title,points_r1_2021
code,,
AL801,SoftwareDesignforVirtualRealityandGaming,300.0
AL802,SoftwareDesigninArtificialIntelligenceforClo,313.0
AL803,SoftwareDesignforMobileAppsandConnectedDevi,350.0
AL805,ComputerEngineeringforNetworkInfrastructure,321.0
AL810,QuantitySurveying,328.0
...,...,...
SG441,Environmental Science,NaN
SG446,Applied Archaeology,NaN
TL803,Music Technology,NaN


In [57]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

,code,points_r1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1455,WD200,AQA
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [58]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

,points_r1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD200,AQA
WD210,279
WD211,271


In [59]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

,title,points_r1_2021,points_r1_2020
code,,,
AL801,SoftwareDesignforVirtualRealityandGaming,300.0,303
AL802,SoftwareDesigninArtificialIntelligenceforClo,313.0,332
AL803,SoftwareDesignforMobileAppsandConnectedDevi,350.0,337
AL805,ComputerEngineeringforNetworkInfrastructure,321.0,333
AL810,QuantitySurveying,328.0,319
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


In [60]:
df2019_r1 = df2019[['code', 'points_r1']]
df2019_r1.columns = ['code', 'points_r1_2019']
df2019_r1

,code,points_r1_2019
0,AL801,304
1,AL802,301
2,AL803,309
3,AL805,329
4,AL810,307
...,...,...
925,WD200,221
926,WD210,271
927,WD211,275
928,WD212,274


In [61]:
# Set the index to the code column.
df2019_r1.set_index('code', inplace=True)
df2019_r1

,points_r1_2019
code,
AL801,304
AL802,301
AL803,309
AL805,329
AL810,307
...,...
WD200,221
WD210,271
WD211,275


In [62]:
# Join 2019 points to allcourses.
allcourses = allcourses.join(df2019_r1)
allcourses

,title,points_r1_2021,points_r1_2020,points_r1_2019
code,,,,
AL801,SoftwareDesignforVirtualRealityandGaming,300.0,303,304
AL802,SoftwareDesigninArtificialIntelligenceforClo,313.0,332,301
AL803,SoftwareDesignforMobileAppsandConnectedDevi,350.0,337,309
AL805,ComputerEngineeringforNetworkInfrastructure,321.0,333,329
AL810,QuantitySurveying,328.0,319,307
...,...,...,...,...
SG441,Environmental Science,NaN,NaN,297
SG446,Applied Archaeology,NaN,NaN,289
TL803,Music Technology,NaN,NaN,264


In [64]:
allcourses.loc['WD148']

title             Tourism Marketing
points_r1_2021                  NaN
points_r1_2020                  NaN
points_r1_2019                  269
Name: WD148, dtype: object

In [67]:
allcourses.iloc[1213]

title             Tourism Marketing
points_r1_2021                  NaN
points_r1_2020                  NaN
points_r1_2019                  269
Name: WD148, dtype: object

In [68]:
allcourses.sort_values('code')

,title,points_r1_2021,points_r1_2020,points_r1_2019
code,,,,
AC120,InternationalBusiness,294.0,209,234
AC137,LiberalArts,271.0,252,252
AD101,FirstYearArtandDesign(CommonEntryportfolio),554.0,#+matric,# +matric 550
AD102,GraphicDesignandMovingImageDesign(portfolio),538.0,#+matric,# +matric 635
AD103,TextileandSurfaceDesignandJewelleryandObjec,505.0,#+matric,# +matric 545
...,...,...,...,...
WD211,CreativeComputing,270.0,271,275
WD212,RecreationandSportManagement,262.0,270,274
WD230,MechanicalandManufacturingEngineering,230.0,253,273


## References
***

[01][The official Python Regular Expressions Documentation](https://docs.python.org/3/library/re.html)<br>
[02][Camelot: PDF Table Extraction for Humans](https://camelot-py.readthedocs.io/en/master/)<br>
[03][How to Extract tabular data from PDF document using Camelot in Python](https://www.analyticsvidhya.com/blog/2020/08/how-to-extract-tabular-data-from-pdf-document-using-camelot-in-python/)<br>
[04][Camelot - Quickstart](https://camelot-py.readthedocs.io/en/master/user/quickstart.html)<br>
[05][Camelot - API Documentation](https://camelot-py.readthedocs.io/en/master/api.html)<br>
[06][Ghostscript](https://www.ghostscript.com/releases/gsdnld.html)<br>
[07][pandas.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)<br>
[08][Stackoverflow - Merging two or more continuous tables from a list in Python](https://stackoverflow.com/questions/60082511/merging-two-or-more-continuous-tables-from-a-list-in-python)<br>
[09][Data to Fish - How to Convert Strings to Integers in Pandas DataFrame](https://datatofish.com/string-to-integer-dataframe/)<br>
[10][pandas - pandas.DataFrame.dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html)<br>
[11][GeeksforGeeks - Python | Pandas dataframe.replace()](https://www.geeksforgeeks.org/python-pandas-dataframe-replace/)<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>

***
# End